In [1]:
!pip install transformers

     |████████████████████████████████| 2.9 MB 5.1 MB/s 
     |████████████████████████████████| 596 kB 45.5 MB/s 
     |████████████████████████████████| 3.3 MB 32.9 MB/s 
     |████████████████████████████████| 895 kB 37.3 MB/s 
     |████████████████████████████████| 56 kB 4.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd drive/MyDrive/Finance\ QA/

/content/drive/.shortcut-targets-by-id/1iz5ONPloFwqdHSl7OCOYnLOYblLdLyPZ/Finance QA/bertMLM_Jason


In [5]:
%ls

Untitled0.ipynb


In [6]:
from transformers import BertTokenizer, BertForMaskedLM
import torch

In [7]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
import json
f = open("/content/drive/MyDrive/Finance QA/data/valid_unfiltered_qa_pairs_with_data.json",)
data = json.load(f)

print(len(data))
f.close()

5296


In [10]:
print(data[0].keys())

dict_keys(['title', 'executives', 'analysts', 'expo', 'Q_A_pairs'])


In [11]:
expo = data[0]['expo']
expo.index('Operator')

24

In [12]:
expo_train = []
for d in data:
  start_pos = None
  if 'Operator' in d['expo']:
    start_pos = d['expo'].index('Operator')
  elif 'operator' in d['expo']:
    start_pos = d['expo'].index('operator')
  else:
    continue
  good_d = d['expo'][start_pos+1:]
  for entry in good_d:
    entry = entry.split('. ')
    for sent in entry:
      sent += '.'
      if len(sent)>30:
        expo_train.append(sent)

In [13]:
import re
qa_train = []
for d in data:
  for pair in d['Q_A_pairs']:
    for sent in pair:
      lis = re.split("([?|.] )", sent)
      j = 0
      while  j < len(lis)-1:
        joint = lis[j] + lis[j+1]
        if '\n' in joint:
          joint.split('\n')
          for sent in joint:
            if len(sent) > 30:
              qa_train.append(sent)
        else:
          if len(joint) > 30:
            qa_train.append(joint)
        j = j + 2

In [14]:
print(qa_train[0:20])

['Thank you very much for your time. ', "We had a lot of communication with our corporate accounts, and we've had appropriate questions and concerns, particularly with regard to our corporate accounts, with whom we have a good relationship and we've already communicated with. ", 'And our corporate accounts are largely supportive. ', 'They want us to do the right thing. ', 'But they believe in us and believe that we will get this fixed. ', "I was just actually -- I mean, I don't want to beat this up. ", 'I just was going to change the subject and actually ask you about regional -- regions in the domestic market. ', 'I know you were talking about improvement in various domestic markets, and I wonder if there are any specific areas of improvement that you would call out as being noteworthy. ', 'Well, we saw improvement -- we saw PRASM up, actually, the most in Houston. ', "To some degree, that's -- we have less capacity in Houston. ", "We're looking forward to actually re-banking Houston 

In [15]:
print(len(qa_train))

614132


In [16]:
inputs = tokenizer(qa_train[0:100000], return_tensors='pt', max_length=1024, padding=True, truncation=True)

In [17]:
inputs['labels'] = inputs.input_ids.detach().clone()

In [18]:
rand = torch.rand(inputs.input_ids.shape)

In [19]:
inputs.input_ids

tensor([[  101,  4067,  2017,  ...,     0,     0,     0],
        [  101,  2057,  2018,  ...,     0,     0,     0],
        [  101,  1998,  2256,  ...,     0,     0,     0],
        ...,
        [  101,  2045,  2018,  ...,     0,     0,     0],
        [  101,  3439, 20056,  ...,     0,     0,     0],
        [  101,  2043, 29403,  ...,     0,     0,     0]])

In [20]:
mask_arr = (rand < 0.15)*(inputs.input_ids != 101)*(inputs.input_ids != 102)*(inputs.input_ids != 0)
mask_arr

tensor([[False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        ...,
        [False, False, False,  ..., False, False, False],
        [False,  True,  True,  ..., False, False, False],
        [False, False, False,  ..., False, False, False]])

In [21]:
selection = []
for i in range(mask_arr.shape[0]):
  selection.append(
    torch.flatten(mask_arr[i].nonzero()).tolist()
  )

In [22]:
selection

[[],
 [4, 12, 16, 17, 21, 22, 30, 36, 37, 38],
 [],
 [2, 5, 6],
 [2, 3, 6],
 [2, 11, 18],
 [1, 3, 17],
 [6, 9, 17],
 [1, 6, 12, 15, 17, 20],
 [2, 5, 12, 13, 16],
 [13, 14, 23, 25, 26],
 [2, 7, 15, 17, 31, 47, 52, 53],
 [1, 6],
 [20],
 [11],
 [6, 7, 11],
 [10, 19, 22, 33, 35],
 [2],
 [7, 18, 19, 22, 31, 32, 43, 50],
 [12, 14],
 [6, 9],
 [1, 3, 10, 11, 20, 25, 26, 35, 40, 45, 53],
 [5, 9],
 [6, 15, 16, 18],
 [5, 6],
 [2, 6, 7, 22, 25],
 [1, 6],
 [],
 [4, 10, 21, 25],
 [11, 13, 20, 32, 34, 35, 37],
 [1, 7, 13, 15, 21, 28, 29],
 [7, 16, 18, 23],
 [4, 8, 12, 14, 20, 21, 23, 27, 28, 46],
 [4, 7, 9, 11, 20, 28, 31, 37],
 [2],
 [4, 7, 8],
 [],
 [6, 12, 19, 20, 23, 37, 41, 42, 47, 52, 55],
 [2, 3],
 [7, 16, 23, 25, 26, 29, 38, 41, 48, 50, 64, 69],
 [7, 17, 18],
 [2, 12, 14],
 [3, 16, 26, 31],
 [6, 7, 10, 25, 31, 33, 36, 38, 39, 53, 54, 55],
 [1, 3, 6, 28],
 [],
 [16, 25, 28],
 [11, 19, 31, 32, 37],
 [],
 [4, 12, 16],
 [7, 12, 13, 20],
 [2, 8],
 [1],
 [15, 17],
 [12],
 [8, 9, 19, 24, 30, 32],
 [

In [23]:
for i in range(inputs.input_ids.shape[0]):
    inputs.input_ids[i, selection[i]] = 103

In [24]:
inputs.input_ids

tensor([[  101,  4067,  2017,  ...,     0,     0,     0],
        [  101,  2057,  2018,  ...,     0,     0,     0],
        [  101,  1998,  2256,  ...,     0,     0,     0],
        ...,
        [  101,  2045,  2018,  ...,     0,     0,     0],
        [  101,   103,   103,  ...,     0,     0,     0],
        [  101,  2043, 29403,  ...,     0,     0,     0]])

In [25]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# and move our model over to the selected device
model.to(device)
# activate training mode
model.train()

from transformers import AdamW
# initialize optimizer
optim = AdamW(model.parameters(), lr=5e-5)

In [26]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

In [27]:
dataset = Dataset(inputs)

In [28]:
loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=False)

In [34]:
from tqdm.notebook import tqdm # for our progress bar

epochs = 2

for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(loader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all tensor batches required for training
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        # process
        outputs = model(input_ids, attention_mask=attention_mask,
                        labels=labels)
        # extract loss
        loss = outputs.loss
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

  0%|          | 0/6250 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """


  0%|          | 0/6250 [00:00<?, ?it/s]

In [31]:
%pwd

'/content/drive/.shortcut-targets-by-id/1iz5ONPloFwqdHSl7OCOYnLOYblLdLyPZ/Finance QA'

In [35]:
WEIGHT_PATH = 'bertMLM_Jason/weights/bertMLM'
model.save_pretrained(WEIGHT_PATH)